## 神经网络反向传播

### 神经网络学习的整个流程
主要有以下4个步骤：

- 前提：神经网络已经有合适的权重和偏置值。
- 选取输入数据：mini-batch从数据中随机选取一部分数据进行训练
- 计算梯度：计算各个权重参数的梯度
- 更新参数：将权重参数沿着梯度方向进行更新
- 重复上面的1-3步

这里我们使用误差反向传播的方案来进行梯度计算，可以大大提高学习的效率。

### 神经网络反向传播的实践

这里接着上一章节来学习，两层神经网络的实践，主要涉及到变量如下：
- params: 保存神经网络的参数的字典型变量
    - params['W1']  是第 1 层的权重，
    - params['b1']  是第 1 层的偏置。
    - params['W2']  是第 2 层的权重， 
    - params['b2']  是第 2层的偏置
- layers:保存神经网络的层的有序字典型变量。
    - 以  layers['Affine1'] 、 
    - layers['ReLu1'] 、
    - layers['Affine2']的形式，通过有序字典保存各个层
    
主要的方法：
- __init__(self, input_size, hidden_size, output_size, weight_init_std)进行初始化
- predict(self, x)进行识别
- loss(self, x, t) 计算损失函数的值。
- accuracy(self, x, t) 计算识别精度
- gradient(self, x, t) 通过误差反向传播法计算关于权重参数的梯度

下面coding 实现如下：


In [ ]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self,input_size,hidden_size,
                 out_size,weight_init_std=0.01):
               # 初始化权重
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
         # 生成层
        self.layers = OrderedDict()
        
         self.layers['Affine1'] = \
            Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = \
            Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
       # x:输入数据, t:监督数据
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def gradient(self, x, t):
        # forward
        self.loss(x, t)
        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        # 设定
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        return grads
            